<h1 align='center'>Automated weather data visualization</h1>

<h2 align='center'>About the data</h2>

WeatherAPI.com is a powerful fully managed free weather and geolocation API provider that provides extensive APIs that range from the weather forecast, historical weather, future weather, weather alerts, air quality data, IP lookup, and astronomy through to sports, time zone, and geolocation.

The data contains: 
- Real-time weather
- Up to 14 day hourly and 15 min weather forecast
- Historical hourly and 15 min weather forecast (from 1-Jan-2010 onwards)
- Future Weather (Upto 365 days ahead)
- Global Weather Alerts
- Air Quality Data
- Astronomy
- Time Zone
- IP Lookup
- Sports API
- Location data
- Up to 99.99% uptime

In [1]:
import requests
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import ipywidgets as widgets

In [ ]:
def extract_weather_by_lat_lon(api_key,lat, lon):

    try:
        # Perform call
        url = "https://weatherapi-com.p.rapidapi.com/forecast.json"

        querystring = {"q":f"5{lat},{lon}","days":"5"}

        headers = {
            "X-RapidAPI-Key": api_key,
            "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(e.response.text)
        return {}
    
def transform_json_to_dataframe(response):
    try:

        unnested_df = pd.json_normalize(response, record_path=['forecast','forecastday','hour'], meta=['location','current'])
        unnested_df.drop(columns=['location','current'], inplace=True)

        location_df = pd.json_normalize(response['location'])


        for col_name in location_df.columns:
            unnested_df[col_name] = location_df[col_name][0]

        return unnested_df

    except KeyError as e:
        print("Key Error:", e)
        return pd.DataFrame()
    except Exception as e:
        print("Other Error:", e)
        return pd.DataFrame()
    

def extraction_df_lat_lon(api_key,lat, lon):
    response = extract_weather_by_lat_lon(api_key,lat, lon)
    return transform_json_to_dataframe(response)



In [2]:
# Load api key
load_dotenv()
api_key = os.getenv("RapidAPI")

In [26]:
# Perform call
url = "https://weatherapi-com.p.rapidapi.com/forecast.json"

querystring = {"q":"53.1,-0.13","days":"5"}

headers = {
	"X-RapidAPI-Key": api_key,
	"X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

In [27]:
# Wrangle data into dataframes 

unnested_df = pd.json_normalize(response.json(), record_path=['forecast','forecastday','hour'], meta=['location','current'])
unnested_df.drop(columns=['location','current'], inplace=True)

location_df = pd.json_normalize(response.json()['location'])


for col_name in location_df.columns:
    unnested_df[col_name] = location_df[col_name][0]



In [28]:
fig = px.scatter_geo(unnested_df, lat="lat",lon="lon", color="region",
                     hover_name="country", size="wind_kph",
                     animation_frame="time",
                     projection="natural earth")

fig.show()

In [10]:
px.scatter_geo(unnested_df, 
               lat="lat", 
               lon="lon", 
               color="temp_c", 
               hover_data=["time","temp_c","windchill_c"], 
               animation_frame="time",
               title="Temperature in Boston,London",
               )


In [9]:
fig = px.scatter(unnested_df,
           x="time",
           y="temp_c",
           color="windchill_c",
           hover_name="name",
           hover_data=["country","region","time","temp_c","windchill_c"],
           title="Temperature vs Windchill forecast")

fig.show()